# Neon Runner — Advanced (Levels + Moving Platforms + Dash/Double-Jump/Wall-Jump + Final Boss)

This notebook contains the **full game code**.

### Controls
- **A / D**: move
- **W / SPACE**: jump (supports **coyote time + jump buffer**)
- **Wall-jump**: jump while touching a wall in the air
- **Double-jump**: one extra jump while airborne
- **Left Shift**: **dash**
- **J**: shoot
- **ESC**: pause

### Level pipeline (external files)
- Levels are loaded from `levels/*.json` by default (created automatically if missing).
- Optional: if you install `pytmx`, you can also load `.tmx` levels (basic support).

> **Note:** Pygame opens a separate window. Run this notebook locally (Anaconda/Jupyter/VS Code) for best results.


In [4]:
# If needed, install dependencies (run once)
# !pip install pygame
# Optional TMX loader:
# !pip install pytmx


In [5]:
import os, json

os.makedirs("levels", exist_ok=True)

# --- Sample levels (JSON) ---
# Legend in grid strings:
#  # = solid tile
#  . = empty
#  P = player start
#  E = enemy
#  C = coin
#  X = spikes
#  D = door (exit)
#  B = boss spawn (final stage)

SAMPLE_LEVELS = [
    {
        "name": "Stage 1",
        "tile_size": 40,
        "grid": [
            "########################################",
            "#.............C..............C........#",
            "#..............######..................#",
            "#...P............................E.....#",
            "#..........#######.....................#",
            "#..............................#####...#",
            "#.....C......................C.........#",
            "#..................E...................#",
            "#..............######..............D...#",
            "#..............#....#..................#",
            "#.....X........#....#........X.........#",
            "########################################",
        ],
        "moving_platforms": [
            {"x": 10, "y": 6, "w": 3, "h": 1, "path": [[10, 6], [18, 6]], "speed": 2.5},
        ],
        "door": {"requires_coins": True, "requires_boss": False},
    },
    {
        "name": "Stage 2",
        "tile_size": 40,
        "grid": [
            "########################################",
            "#..C..............#..............C.....#",
            "#..........E......#....................#",
            "#........#####....#....#####...........#",
            "#..P.....#........#........#...........#",
            "#........#....C...#...C....#.....E.....#",
            "#........#........#........#...........#",
            "#........#####....#....#####........D..#",
            "#..................#...................#",
            "#....X.............#..........X........#",
            "########################################",
        ],
        "moving_platforms": [
            {"x": 6, "y": 4, "w": 3, "h": 1, "path": [[6, 4], [6, 7]], "speed": 2.0},
            {"x": 26, "y": 5, "w": 3, "h": 1, "path": [[26, 5], [34, 5]], "speed": 3.0},
        ],
        "door": {"requires_coins": True, "requires_boss": False},
    },
    {
        "name": "Stage 3 (Boss)",
        "tile_size": 40,
        "grid": [
            "########################################",
            "#..C....######..............######..C..#",
            "#.......#....#..............#....#.....#",
            "#...E...#....#....C....C....#....#..E..#",
            "#.......#....######....######....#.....#",
            "#..P....#........................#..B..#",
            "#.......######....X......X....######...#",
            "#..................######..............#",
            "#..C...........E....#..#....E......C.D.#",
            "#...................#..#...............#",
            "########################################",
        ],
        "moving_platforms": [
            {"x": 15, "y": 7, "w": 4, "h": 1, "path": [[15, 7], [22, 7]], "speed": 2.5},
            {"x": 30, "y": 2, "w": 3, "h": 1, "path": [[30, 2], [30, 5]], "speed": 2.0},
        ],
        "door": {"requires_coins": True, "requires_boss": True},
        "boss": {"hp": 45},
    },
]

def ensure_sample_levels():
    for i, data in enumerate(SAMPLE_LEVELS, start=1):
        path = os.path.join("levels", f"level{i}.json")
        if not os.path.exists(path):
            with open(path, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
    print("Levels ready in ./levels (edit them to create new stages).")

ensure_sample_levels()


Levels ready in ./levels (edit them to create new stages).


In [ ]:
import math
import random
import sys
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict, Any

import pygame

# =========================
# Neon Runner — Advanced
# =========================
# Adds:
# - External level loading (JSON, optional TMX via pytmx)
# - Moving platforms
# - Dash
# - Double-jump
# - Wall-jump
# - Final boss fight with bullets + phases
# - Higher jump (tuned)
#
# Run locally: this opens a pygame window.

# ----------- Config -----------
WIDTH, HEIGHT = 960, 540
FPS = 60

BG = (10, 10, 16)
WHITE = (235, 235, 245)
NEON = (80, 255, 210)
PINK = (255, 100, 200)
YELLOW = (255, 220, 80)
RED = (255, 80, 80)
BLUE = (120, 180, 255)
GRAY = (80, 90, 110)

def clamp(v, a, b):
    return a if v < a else b if v > b else v

def lerp(a, b, t):
    return a + (b - a) * t

def sign(x):
    return -1 if x < 0 else 1 if x > 0 else 0

# ----------- Camera -----------
@dataclass
class Camera:
    x: float = 0
    y: float = 0
    shake: float = 0

    def update(self, target_rect, world_w, world_h, dt):
        tx = target_rect.centerx - WIDTH / 2
        ty = target_rect.centery - HEIGHT / 2
        tx = clamp(tx, 0, max(0, world_w - WIDTH))
        ty = clamp(ty, 0, max(0, world_h - HEIGHT))

        self.x = lerp(self.x, tx, 1 - math.pow(0.001, dt))
        self.y = lerp(self.y, ty, 1 - math.pow(0.001, dt))
        self.shake = max(0, self.shake - 18 * dt)

    def apply(self, rect):
        ox, oy = 0, 0
        if self.shake > 0:
            ox = random.uniform(-self.shake, self.shake)
            oy = random.uniform(-self.shake, self.shake)
        return rect.move(-self.x + ox, -self.y + oy)

# ----------- Particle system -----------
@dataclass
class Particle:
    x: float
    y: float
    vx: float
    vy: float
    life: float
    radius: float
    color: tuple

class Particles:
    def __init__(self):
        self.items: List[Particle] = []

    def burst(self, x, y, n=14, color=NEON, speed=240, radius=3):
        for _ in range(n):
            ang = random.uniform(0, math.tau)
            sp = random.uniform(speed * 0.35, speed)
            vx = math.cos(ang) * sp
            vy = math.sin(ang) * sp
            life = random.uniform(0.25, 0.75)
            r = random.uniform(1.5, radius)
            self.items.append(Particle(x, y, vx, vy, life, r, color))

    def update(self, dt):
        alive = []
        for p in self.items:
            p.life -= dt
            if p.life > 0:
                p.vy += 600 * dt
                p.x += p.vx * dt
                p.y += p.vy * dt
                alive.append(p)
        self.items = alive

    def draw(self, surf, cam: Camera):
        for p in self.items:
            rr = pygame.Rect(p.x - p.radius, p.y - p.radius, p.radius * 2, p.radius * 2)
            rr = cam.apply(rr)
            pygame.draw.circle(surf, p.color, rr.center, max(1, int(p.radius)))

# ----------- Moving platform -----------
class MovingPlatform:
    def __init__(self, x, y, w, h, path_pts, speed_px):
        self.rect = pygame.Rect(x, y, w, h)
        self.prev = self.rect.copy()
        self.path = path_pts[:]  # list of (x,y) in pixels
        self.speed = speed_px
        self.t = 0.0
        self.seg = 0
        self.forward = True

    def update(self, dt):
        self.prev = self.rect.copy()
        if len(self.path) < 2:
            return

        a = self.path[self.seg]
        b = self.path[(self.seg + 1) % len(self.path)]
        ax, ay = a
        bx, by = b
        dx, dy = bx - ax, by - ay
        dist = math.hypot(dx, dy) or 1.0
        vx, vy = dx / dist, dy / dist

        move = self.speed * dt
        self.t += move

        while self.t >= dist:
            self.t -= dist
            self.seg = (self.seg + 1) % len(self.path)
            a = self.path[self.seg]
            b = self.path[(self.seg + 1) % len(self.path)]
            ax, ay = a
            bx, by = b
            dx, dy = bx - ax, by - ay
            dist = math.hypot(dx, dy) or 1.0
            vx, vy = dx / dist, dy / dist

        nx = ax + vx * self.t
        ny = ay + vy * self.t
        self.rect.topleft = (int(nx), int(ny))

    @property
    def delta(self):
        return (self.rect.x - self.prev.x, self.rect.y - self.prev.y)

    def draw(self, surf, cam: Camera):
        rr = cam.apply(self.rect)
        pygame.draw.rect(surf, (28, 34, 52), rr, border_radius=10)
        pygame.draw.rect(surf, BLUE, rr, 2, border_radius=10)

# ----------- Level loading (JSON / optional TMX) -----------
class Level:
    def __init__(self, data: Dict[str, Any]):
        self.name = data.get("name", "Stage")
        self.tile_size = int(data.get("tile_size", 40))
        self.grid: List[str] = data.get("grid", [])
        self.h = len(self.grid)
        self.w = len(self.grid[0]) if self.h else 0
        self.world_w = max(1, self.w * self.tile_size)
        self.world_h = max(1, self.h * self.tile_size)

        self.solids = set()
        self.spikes = set()
        self.coins = set()
        self.enemies_spawn = []
        self.player_spawn = (1, 1)
        self.door_pos = None
        self.boss_pos = None

        for y, row in enumerate(self.grid):
            for x, ch in enumerate(row):
                if ch == "#":
                    self.solids.add((x, y))
                elif ch == "X":
                    self.spikes.add((x, y))
                elif ch == "C":
                    self.coins.add((x, y))
                elif ch == "E":
                    self.enemies_spawn.append((x, y))
                elif ch == "P":
                    self.player_spawn = (x, y)
                elif ch == "D":
                    self.door_pos = (x, y)
                elif ch == "B":
                    self.boss_pos = (x, y)

        self.total_coins = len(self.coins)
        self.door_rules = data.get("door", {"requires_coins": True, "requires_boss": False})
        self.boss_cfg = data.get("boss", {"hp": 40})

        # moving platforms: positions and path are in TILE coordinates in JSON
        self.platforms: List[MovingPlatform] = []
        for p in data.get("moving_platforms", []):
            tx, ty = p["x"], p["y"]
            tw, th = p.get("w", 3), p.get("h", 1)
            wpx, hpx = tw * self.tile_size, th * self.tile_size
            path = p.get("path", [[tx, ty], [tx + 4, ty]])
            path_px = [(pt[0] * self.tile_size, pt[1] * self.tile_size) for pt in path]
            speed_tiles = float(p.get("speed", 2.0))  # tiles/sec
            speed_px = speed_tiles * self.tile_size
            self.platforms.append(MovingPlatform(tx * self.tile_size, ty * self.tile_size, wpx, hpx, path_px, speed_px))

    def tile_rect(self, tx, ty):
        s = self.tile_size
        return pygame.Rect(tx * s, ty * s, s, s)

    def solid_rects_near(self, rect: pygame.Rect):
        s = self.tile_size
        x0 = max(0, rect.left // s - 1)
        x1 = min(self.w - 1, rect.right // s + 1)
        y0 = max(0, rect.top // s - 1)
        y1 = min(self.h - 1, rect.bottom // s + 1)

        out = []
        for ty in range(y0, y1 + 1):
            for tx in range(x0, x1 + 1):
                if (tx, ty) in self.solids:
                    out.append(self.tile_rect(tx, ty))
        # platforms (few, so just cull by inflate)
        inflated = rect.inflate(200, 200)
        for p in self.platforms:
            if inflated.colliderect(p.rect):
                out.append(p.rect)
        return out

    def spike_rects_near(self, rect: pygame.Rect):
        s = self.tile_size
        x0 = max(0, rect.left // s - 1)
        x1 = min(self.w - 1, rect.right // s + 1)
        y0 = max(0, rect.top // s - 1)
        y1 = min(self.h - 1, rect.bottom // s + 1)

        out = []
        for ty in range(y0, y1 + 1):
            for tx in range(x0, x1 + 1):
                if (tx, ty) in self.spikes:
                    r = self.tile_rect(tx, ty)
                    r = pygame.Rect(r.x + 6, r.y + 14, r.w - 12, r.h - 14)
                    out.append(r)
        return out

    def coin_rects_near(self, rect: pygame.Rect):
        s = self.tile_size
        x0 = max(0, rect.left // s - 1)
        x1 = min(self.w - 1, rect.right // s + 1)
        y0 = max(0, rect.top // s - 1)
        y1 = min(self.h - 1, rect.bottom // s + 1)

        out = []
        for ty in range(y0, y1 + 1):
            for tx in range(x0, x1 + 1):
                if (tx, ty) in self.coins:
                    r = self.tile_rect(tx, ty)
                    rr = pygame.Rect(r.centerx - 10, r.centery - 10, 20, 20)
                    out.append(((tx, ty), rr))
        return out

    def door_rect(self):
        if not self.door_pos:
            return None
        tx, ty = self.door_pos
        r = self.tile_rect(tx, ty)
        return pygame.Rect(r.x + 6, r.y + 4, r.w - 12, r.h - 8)

    def boss_spawn_px(self):
        if not self.boss_pos:
            return None
        tx, ty = self.boss_pos
        r = self.tile_rect(tx, ty)
        return (r.centerx, r.centery)

    def update(self, dt):
        for p in self.platforms:
            p.update(dt)

    def draw(self, surf, cam: Camera, door_open: bool, boss_alive: bool):
        view = pygame.Rect(int(cam.x), int(cam.y), WIDTH, HEIGHT)
        s = self.tile_size
        tx0 = max(0, view.left // s - 1)
        tx1 = min(self.w - 1, view.right // s + 1)
        ty0 = max(0, view.top // s - 1)
        ty1 = min(self.h - 1, view.bottom // s + 1)

        # tiles
        for ty in range(ty0, ty1 + 1):
            for tx in range(tx0, tx1 + 1):
                if (tx, ty) in self.solids:
                    r = self.tile_rect(tx, ty)
                    rr = cam.apply(r)
                    pygame.draw.rect(surf, (26, 30, 44), rr, border_radius=8)
                    pygame.draw.rect(surf, (40, 46, 64), rr, 2, border_radius=8)

                if (tx, ty) in self.spikes:
                    r = self.tile_rect(tx, ty)
                    rr = cam.apply(r)
                    base_y = rr.bottom - 6
                    for i in range(4):
                        x0 = rr.left + 4 + i * (rr.w - 8) / 4
                        x1 = rr.left + 4 + (i + 1) * (rr.w - 8) / 4
                        mid = (x0 + x1) / 2
                        pts = [(x0, base_y), (mid, rr.top + 14), (x1, base_y)]
                        pygame.draw.polygon(surf, RED, pts)

        # moving platforms
        for p in self.platforms:
            p.draw(surf, cam)

        # coins
        for (tx, ty) in list(self.coins):
            r = self.tile_rect(tx, ty)
            rr = cam.apply(r)
            pygame.draw.circle(surf, YELLOW, rr.center, 9)
            pygame.draw.circle(surf, WHITE, rr.center, 9, 2)

        # boss marker (if any and alive, subtle)
        if self.boss_pos and boss_alive:
            bx, by = self.boss_spawn_px()
            rr = cam.apply(pygame.Rect(bx - 10, by - 10, 20, 20))
            pygame.draw.circle(surf, (160, 90, 255), rr.center, 10, 2)

        # door
        if self.door_pos:
            dr = self.door_rect()
            drr = cam.apply(dr)
            color = NEON if door_open else GRAY
            pygame.draw.rect(surf, color, drr, border_radius=10)
            pygame.draw.rect(surf, WHITE, drr, 2, border_radius=10)
            lock_r = pygame.Rect(drr.centerx - 8, drr.centery - 8, 16, 16)
            pygame.draw.circle(surf, (10, 10, 16), lock_r.center, 6)
            if door_open:
                pygame.draw.circle(surf, WHITE, lock_r.center, 6, 2)

# Optional TMX loader (basic)
def load_level_from_tmx(path: str) -> Dict[str, Any]:
    try:
        from pytmx.util_pygame import load_pygame
    except Exception as e:
        raise RuntimeError("TMX support requires: pip install pytmx") from e

    tmx = load_pygame(path)
    tile_size = tmx.tilewidth
    w, h = tmx.width, tmx.height

    # Build an empty grid and fill solids/spikes/coins by tile properties if present.
    grid = [["." for _ in range(w)] for _ in range(h)]

    def set_char(x, y, ch):
        if 0 <= x < w and 0 <= y < h:
            grid[y][x] = ch

    # Tile layers: inspect properties (solid/spike/coin).
    for layer in tmx.visible_layers:
        if hasattr(layer, "tiles"):
            for x, y, gid in layer.tiles():
                props = tmx.get_tile_properties_by_gid(gid) or {}
                if props.get("solid"):
                    set_char(x, y, "#")
                if props.get("spike"):
                    set_char(x, y, "X")
                if props.get("coin"):
                    set_char(x, y, "C")

    # Object layer conventions:
    # - object type/name "Player" => player spawn
    # - "Enemy" => enemy spawns
    # - "Door" => door
    # - "Boss" => boss spawn
    # - "Platform" => moving platform with custom properties: path (as "x1,y1;x2,y2;...") and speed (tiles/sec)
    platforms = []
    door = {"requires_coins": True, "requires_boss": False}
    boss = {"hp": 45}
    for layer in tmx.visible_layers:
        if getattr(layer, "objects", None) is None:
            continue
        for obj in layer:
            n = (obj.name or obj.type or "").lower()
            tx = int(obj.x // tile_size)
            ty = int(obj.y // tile_size)
            if "player" in n:
                set_char(tx, ty, "P")
            elif "enemy" in n:
                set_char(tx, ty, "E")
            elif "door" in n:
                set_char(tx, ty, "D")
                door["requires_coins"] = bool(obj.properties.get("requires_coins", True))
                door["requires_boss"] = bool(obj.properties.get("requires_boss", False))
            elif "boss" in n:
                set_char(tx, ty, "B")
                boss["hp"] = int(obj.properties.get("hp", boss["hp"]))
            elif "platform" in n:
                w_tiles = max(1, int(obj.width // tile_size))
                h_tiles = max(1, int(obj.height // tile_size))
                speed = float(obj.properties.get("speed", 2.5))
                path_str = str(obj.properties.get("path", f"{tx},{ty};{tx+6},{ty}"))
                pts = []
                for part in path_str.split(";"):
                    xs, ys = part.split(",")
                    pts.append([int(xs), int(ys)])
                platforms.append({"x": tx, "y": ty, "w": w_tiles, "h": h_tiles, "path": pts, "speed": speed})

    return {
        "name": os.path.basename(path),
        "tile_size": tile_size,
        "grid": ["".join(row) for row in grid],
        "moving_platforms": platforms,
        "door": door,
        "boss": boss,
    }

def load_level(path: str) -> Level:
    if path.lower().endswith(".tmx"):
        data = load_level_from_tmx(path)
        return Level(data)
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return Level(data)

# ----------- Entities -----------
class Bullet:
    def __init__(self, x, y, vx, vy, owner="player", dmg=1, life=1.2, color=PINK):
        self.x, self.y = float(x), float(y)
        self.vx, self.vy = float(vx), float(vy)
        self.owner = owner
        self.dmg = dmg
        self.r = 5
        self.life = float(life)
        self.color = color

    @property
    def rect(self):
        return pygame.Rect(int(self.x - self.r), int(self.y - self.r), self.r * 2, self.r * 2)

    def update(self, dt, level: Level):
        self.life -= dt
        self.x += self.vx * dt
        self.y += self.vy * dt
        # collide with solids
        for sr in level.solid_rects_near(self.rect):
            if self.rect.colliderect(sr):
                self.life = 0
                break

    def draw(self, surf, cam: Camera):
        rr = cam.apply(self.rect)
        pygame.draw.circle(surf, self.color, rr.center, self.r)
        pygame.draw.circle(surf, WHITE, rr.center, self.r, 1)

class Enemy:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 30, 34)
        self.vx = random.choice([-1, 1]) * 110
        self.vy = 0.0
        self.hp = 3
        self.hit_flash = 0.0

    def update(self, dt, level: Level):
        self.hit_flash = max(0, self.hit_flash - dt * 8)
        self.vy += 1100 * dt
        self.vy = min(self.vy, 1200)

        self.rect.x += int(self.vx * dt)
        for s in level.solid_rects_near(self.rect):
            if self.rect.colliderect(s):
                if self.vx > 0:
                    self.rect.right = s.left
                else:
                    self.rect.left = s.right
                self.vx *= -1
                break

        # edge turn (tile-based-ish heuristic)
        ahead = self.rect.move(int(sign(self.vx) * 20), 4)
        ground_check = ahead.move(0, 28)
        on_something = False
        for s in level.solid_rects_near(ground_check):
            if ground_check.colliderect(s):
                on_something = True
                break
        if not on_something:
            self.vx *= -1

        self.rect.y += int(self.vy * dt)
        for s in level.solid_rects_near(self.rect):
            if self.rect.colliderect(s):
                if self.vy > 0:
                    self.rect.bottom = s.top
                else:
                    self.rect.top = s.bottom
                self.vy = 0
                break

    def damage(self, amount=1):
        self.hp -= amount
        self.hit_flash = 1.0

    def draw(self, surf, cam: Camera):
        rr = cam.apply(self.rect)
        col = (255, 140, 140) if self.hit_flash > 0 else (200, 70, 90)
        pygame.draw.rect(surf, col, rr, border_radius=10)
        pygame.draw.rect(surf, WHITE, rr, 2, border_radius=10)
        pygame.draw.circle(surf, (10, 10, 16), (rr.centerx - 6, rr.centery - 2), 3)
        pygame.draw.circle(surf, (10, 10, 16), (rr.centerx + 6, rr.centery - 2), 3)

class Boss:
    def __init__(self, x, y, hp=45):
        self.rect = pygame.Rect(int(x) - 44, int(y) - 52, 88, 96)
        self.vx = 120.0
        self.vy = 0.0
        self.hp = hp
        self.max_hp = hp
        self.hit_flash = 0.0
        self.cool = 0.0
        self.jump_cd = 1.4
        self.phase = 1
        self.alive = True

    def update(self, dt, level: Level, bullets_out: List[Bullet], player_rect: pygame.Rect):
        if not self.alive:
            return
        self.hit_flash = max(0, self.hit_flash - dt * 6)

        # phases
        hp_ratio = self.hp / max(1, self.max_hp)
        self.phase = 2 if hp_ratio < 0.55 else 1

        # movement: patrol + occasional hop
        self.vy += 1200 * dt
        self.vy = min(self.vy, 1400)

        self.rect.x += int(self.vx * dt)
        hit_wall = False
        for s in level.solid_rects_near(self.rect):
            if self.rect.colliderect(s):
                if self.vx > 0:
                    self.rect.right = s.left
                else:
                    self.rect.left = s.right
                self.vx *= -1
                hit_wall = True
                break

        self.rect.y += int(self.vy * dt)
        on_ground = False
        for s in level.solid_rects_near(self.rect):
            if self.rect.colliderect(s):
                if self.vy > 0:
                    self.rect.bottom = s.top
                    on_ground = True
                else:
                    self.rect.top = s.bottom
                self.vy = 0
                break

        self.jump_cd -= dt
        if on_ground and self.jump_cd <= 0:
            self.jump_cd = 1.3 if self.phase == 1 else 0.95
            # jump toward player a bit
            toward = sign(player_rect.centerx - self.rect.centerx) or random.choice([-1, 1])
            self.vx = toward * (170 if self.phase == 1 else 220)
            self.vy = -520 if self.phase == 1 else -620

        # attack patterns
        self.cool -= dt
        if self.cool <= 0:
            if self.phase == 1:
                self.cool = 0.85
                self._shoot_fan(bullets_out, player_rect, count=5, spread=0.45, speed=420, color=(170, 90, 255))
            else:
                self.cool = 0.55
                # tighter, faster + occasional ring
                if random.random() < 0.25:
                    self._shoot_ring(bullets_out, count=10, speed=360, color=(170, 90, 255))
                else:
                    self._shoot_fan(bullets_out, player_rect, count=7, spread=0.55, speed=520, color=(190, 120, 255))

    def _shoot_fan(self, bullets_out, player_rect, count=5, spread=0.45, speed=420, color=(170, 90, 255)):
        ox, oy = self.rect.centerx, self.rect.centery - 10
        dx = player_rect.centerx - ox
        dy = player_rect.centery - oy
        base = math.atan2(dy, dx)
        for i in range(count):
            t = (i - (count - 1) / 2) / max(1, (count - 1) / 2)
            ang = base + t * spread
            vx = math.cos(ang) * speed
            vy = math.sin(ang) * speed
            bullets_out.append(Bullet(ox, oy, vx, vy, owner="boss", dmg=1, life=2.4, color=color))

    def _shoot_ring(self, bullets_out, count=10, speed=360, color=(170, 90, 255)):
        ox, oy = self.rect.centerx, self.rect.centery - 10
        for i in range(count):
            ang = (i / count) * math.tau
            vx = math.cos(ang) * speed
            vy = math.sin(ang) * speed
            bullets_out.append(Bullet(ox, oy, vx, vy, owner="boss", dmg=1, life=2.0, color=color))

    def damage(self, amount=1):
        if not self.alive:
            return
        self.hp -= amount
        self.hit_flash = 1.0
        if self.hp <= 0:
            self.hp = 0
            self.alive = False

    def draw(self, surf, cam: Camera):
        if not self.alive:
            return
        rr = cam.apply(self.rect)
        col = (200, 120, 255) if self.hit_flash <= 0 else (255, 210, 255)
        pygame.draw.rect(surf, col, rr, border_radius=18)
        pygame.draw.rect(surf, WHITE, rr, 2, border_radius=18)
        # face
        eye_y = rr.y + 36
        pygame.draw.circle(surf, (10, 10, 16), (rr.centerx - 16, eye_y), 6)
        pygame.draw.circle(surf, (10, 10, 16), (rr.centerx + 16, eye_y), 6)
        pygame.draw.rect(surf, (10, 10, 16), (rr.centerx - 18, rr.y + 58, 36, 10), border_radius=6)

# ----------- Player -----------
class Player:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 28, 36)
        self.vx = 0.0
        self.vy = 0.0
        self.facing = 1

        # stats
        self.max_hp = 8
        self.hp = self.max_hp
        self.speed = 460.0
        self.jump_v = 690.0   # higher jump (requested)
        self.fire_rate = 0.24
        self.bullet_speed = 820.0

        # movement tech
        self.coyote = 0.0
        self.jump_buf = 0.0
        self.on_ground = False
        self.wall_left = False
        self.wall_right = False

        self.extra_jumps_max = 1   # double-jump (one extra)
        self.extra_jumps = self.extra_jumps_max

        # dash
        self.dash_cd = 0.0
        self.dash_time = 0.0
        self.dash_speed = 980.0

        # combat
        self.shoot_cd = 0.0
        self.invuln = 0.0

        # platform riding
        self.on_platform: Optional[MovingPlatform] = None

    def hurt(self, dmg=1):
        if self.invuln > 0:
            return False
        self.hp -= dmg
        self.invuln = 0.9
        return True

    def update(self, dt, level: Level, keys, bullets_out: List[Bullet], particles: Particles):
        self.invuln = max(0, self.invuln - dt)
        self.shoot_cd = max(0, self.shoot_cd - dt)
        self.jump_buf = max(0, self.jump_buf - dt)
        self.dash_cd = max(0, self.dash_cd - dt)
        self.wall_left = self.wall_right = False
        self.on_platform = None

        # input
        ax = 0.0
        if keys[pygame.K_a]:
            ax -= 1
        if keys[pygame.K_d]:
            ax += 1
        if ax != 0:
            self.facing = sign(ax)

        # jump buffer
        if keys[pygame.K_w] or keys[pygame.K_SPACE]:
            self.jump_buf = 0.12

        # dash
        dash_pressed = keys[pygame.K_LSHIFT] or keys[pygame.K_RSHIFT]
        if dash_pressed and self.dash_cd <= 0 and self.dash_time <= 0:
            self.dash_time = 0.16
            self.dash_cd = 0.75
            d = self.facing if ax == 0 else sign(ax)
            if d == 0:
                d = self.facing
            self.vx = d * self.dash_speed
            self.vy *= 0.2
            particles.burst(self.rect.centerx, self.rect.centery, n=16, color=BLUE, speed=340, radius=3)

        # shoot
        if keys[pygame.K_j] and self.shoot_cd <= 0:
            self.shoot_cd = self.fire_rate
            bx = self.rect.centerx + self.facing * 18
            by = self.rect.centery - 2
            bullets_out.append(Bullet(bx, by, self.facing * self.bullet_speed, 0, owner="player", dmg=1, life=1.4, color=PINK))
            particles.burst(bx, by, n=6, color=PINK, speed=180, radius=2.5)

        # coyote time
        if self.on_ground:
            self.coyote = 0.11
        else:
            self.coyote = max(0, self.coyote - dt)

        # dash state overrides normal accel a bit
        if self.dash_time > 0:
            self.dash_time -= dt
            # reduced gravity while dashing
            self.vy += 650 * dt
            self.vy = min(self.vy, 1200)
        else:
            # physics: accel + friction
            target = ax * self.speed
            self.vx = lerp(self.vx, target, 1 - math.pow(0.0001, dt))
            if ax == 0:
                self.vx = lerp(self.vx, 0, 1 - math.pow(0.00001, dt))
            # gravity
            self.vy += 1300 * dt
            self.vy = min(self.vy, 1300)

        # Jump logic: ground/coyote > wall-jump > double-jump
        if self.jump_buf > 0:
            did_jump = False
            if self.coyote > 0:
                self.jump_buf = 0
                self.coyote = 0
                self.vy = -self.jump_v
                self.extra_jumps = self.extra_jumps_max
                particles.burst(self.rect.centerx, self.rect.bottom, n=12, color=NEON, speed=260, radius=3)
                did_jump = True
            elif (self.wall_left or self.wall_right):
                # wall-jump
                self.jump_buf = 0
                self.vy = -self.jump_v * 0.92
                push = 520.0
                self.vx = push if self.wall_left else -push
                particles.burst(self.rect.centerx, self.rect.centery, n=14, color=NEON, speed=280, radius=3)
                did_jump = True
            elif self.extra_jumps > 0:
                # double-jump
                self.jump_buf = 0
                self.extra_jumps -= 1
                self.vy = -self.jump_v * 0.95
                particles.burst(self.rect.centerx, self.rect.centery, n=12, color=YELLOW, speed=260, radius=3)
                did_jump = True

        # move X
        self.rect.x += int(self.vx * dt)
        for s in level.solid_rects_near(self.rect):
            if self.rect.colliderect(s):
                if self.vx > 0:
                    self.rect.right = s.left
                    self.wall_right = True
                else:
                    self.rect.left = s.right
                    self.wall_left = True
                self.vx = 0
                break

        # move Y
        self.rect.y += int(self.vy * dt)
        self.on_ground = False
        for s in level.solid_rects_near(self.rect):
            if self.rect.colliderect(s):
                if self.vy > 0:
                    self.rect.bottom = s.top
                    self.on_ground = True
                    # detect if standing on platform
                    for p in level.platforms:
                        if p.rect == s:
                            self.on_platform = p
                            break
                else:
                    self.rect.top = s.bottom
                self.vy = 0
                break

        # platform carry: after collisions, apply platform delta
        if self.on_platform is not None:
            dx, dy = self.on_platform.delta
            self.rect.x += dx
            self.rect.y += dy

        # reset extra jumps when grounded
        if self.on_ground:
            self.extra_jumps = self.extra_jumps_max

    def draw(self, surf, cam: Camera):
        rr = cam.apply(self.rect)
        if self.invuln > 0 and int(self.invuln * 20) % 2 == 0:
            return
        pygame.draw.rect(surf, NEON, rr, border_radius=10)
        pygame.draw.rect(surf, WHITE, rr, 2, border_radius=10)
        visor = pygame.Rect(rr.x + 6, rr.y + 10, rr.w - 12, 10)
        pygame.draw.rect(surf, (10, 10, 16), visor, border_radius=6)
        shine = pygame.Rect(visor.x + 2, visor.y + 2, int(visor.w * 0.5), 3)
        pygame.draw.rect(surf, WHITE, shine, border_radius=3)

# ----------- UI helpers -----------
def draw_text(surf, font, text, x, y, color=WHITE, center=False):
    img = font.render(text, True, color)
    r = img.get_rect()
    if center:
        r.center = (x, y)
    else:
        r.topleft = (x, y)
    surf.blit(img, r)
    return r

def draw_hud(surf, font, player: Player, stage_name: str, coins_left: int, boss: Optional[Boss], paused=False):
    pad = 16
    w = 240
    h = 16
    x, y = pad, pad

    pygame.draw.rect(surf, (20, 20, 28), (x, y, w, h), border_radius=8)
    ratio = player.hp / max(1, player.max_hp)
    pygame.draw.rect(surf, RED, (x, y, int(w * ratio), h), border_radius=8)
    pygame.draw.rect(surf, WHITE, (x, y, w, h), 2, border_radius=8)

    draw_text(surf, font, stage_name, x, y + 24, WHITE)
    draw_text(surf, font, f"Coins left: {coins_left}", x, y + 44, YELLOW)
    draw_text(surf, font, "Shift dash • Wall-jump • Double-jump", x, y + 64, (180, 200, 255))

    if boss is not None and boss.alive:
        # boss HP bar
        bw = 360
        bh = 14
        bx = WIDTH // 2 - bw // 2
        by = 16
        pygame.draw.rect(surf, (20, 20, 28), (bx, by, bw, bh), border_radius=8)
        br = boss.hp / max(1, boss.max_hp)
        pygame.draw.rect(surf, (170, 90, 255), (bx, by, int(bw * br), bh), border_radius=8)
        pygame.draw.rect(surf, WHITE, (bx, by, bw, bh), 2, border_radius=8)
        draw_text(surf, font, "BOSS", bx + bw + 12, by - 2, WHITE)

    if paused:
        draw_text(surf, font, "PAUSED (ESC)", WIDTH // 2, 40, WHITE, center=True)

# ----------- Upgrade system -----------
UPGRADES = [
    ("hp", "Refit Armor (+2 max HP, heal to full)"),
    ("speed", "Servo Legs (+speed)"),
    ("jump", "Boost Springs (+jump)"),
    ("firerate", "Overclock Blaster (+fire rate)"),
    ("bulletspeed", "Mag Rail (+bullet speed)"),
    ("dash", "Flux Drive (dash cooldown -20%)"),
]
def pick_3_upgrades(rng: random.Random):
    return rng.sample(UPGRADES, 3)

def apply_upgrade(player: Player, upg_id: str):
    if upg_id == "hp":
        player.max_hp += 2
        player.hp = player.max_hp
    elif upg_id == "speed":
        player.speed += 70
    elif upg_id == "jump":
        player.jump_v += 70
    elif upg_id == "firerate":
        player.fire_rate = max(0.12, player.fire_rate - 0.05)
    elif upg_id == "bulletspeed":
        player.bullet_speed += 120
    elif upg_id == "dash":
        player.dash_cd = max(0, player.dash_cd * 0.8)  # immediate cooldown relief
        # reduce future cooldown by adjusting base (simple approach)
        # (store in attribute for persistence)
        if not hasattr(player, "dash_cd_mult"):
            player.dash_cd_mult = 1.0
        player.dash_cd_mult *= 0.8

# ----------- Main game -----------
def main(level_paths: List[str]):
    pygame.init()
    pygame.display.set_caption("Neon Runner — Advanced")
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()

    font = pygame.font.SysFont("consolas", 18)
    big = pygame.font.SysFont("consolas", 44, bold=True)
    mid = pygame.font.SysFont("consolas", 28, bold=True)

    rng = random.Random()
    particles = Particles()
    cam = Camera()

    # load levels
    levels = [load_level(p) for p in level_paths]
    level_idx = 0
    level = levels[level_idx]

    # spawn player/entities
    ts = level.tile_size
    player = Player(level.player_spawn[0] * ts + 6, level.player_spawn[1] * ts + 2)
    enemies = [Enemy(tx * ts + 6, ty * ts + 4) for (tx, ty) in level.enemies_spawn]
    bullets: List[Bullet] = []
    enemy_bullets: List[Bullet] = []
    boss: Optional[Boss] = None

    coins_left = level.total_coins
    boss_required = bool(level.door_rules.get("requires_boss", False))
    boss_defeated = False

    def spawn_boss_if_present():
        nonlocal boss
        if level.boss_pos is None:
            boss = None
            return
        bx, by = level.boss_spawn_px()
        boss = Boss(bx, by, hp=int(level.boss_cfg.get("hp", 45)))

    spawn_boss_if_present()

    def door_open():
        coins_ok = (coins_left == 0) if level.door_rules.get("requires_coins", True) else True
        boss_ok = (not boss_required) or boss_defeated
        return coins_ok and boss_ok

    def load_level_idx(i):
        nonlocal level, level_idx, coins_left, enemies, bullets, enemy_bullets, boss, boss_required, boss_defeated, cam
        level_idx = i
        level = levels[level_idx]
        ts = level.tile_size
        player.rect.x = level.player_spawn[0] * ts + 6
        player.rect.y = level.player_spawn[1] * ts + 2
        player.vx = player.vy = 0
        player.invuln = 0
        enemies = [Enemy(tx * ts + 6, ty * ts + 4) for (tx, ty) in level.enemies_spawn]
        bullets = []
        enemy_bullets = []
        coins_left = level.total_coins
        boss_required = bool(level.door_rules.get("requires_boss", False))
        boss_defeated = False
        cam.x = cam.y = 0
        spawn_boss_if_present()

    state = "TITLE"  # TITLE, PLAY, UPGRADE, GAMEOVER, WIN
    paused = False
    upgrade_choices = []
    elapsed = 0.0

    while True:
        dt = clock.tick(FPS) / 1000.0
        dt = min(dt, 1 / 30)

        if state == "PLAY" and not paused:
            elapsed += dt

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                raise SystemExit

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE and state == "PLAY":
                    paused = not paused

                if state == "TITLE":
                    if event.key == pygame.K_RETURN:
                        state = "PLAY"
                        paused = False
                        elapsed = 0.0

                elif state == "GAMEOVER":
                    if event.key == pygame.K_r:
                        load_level_idx(0)
                        particles.items.clear()
                        player.hp = player.max_hp
                        state = "PLAY"
                        paused = False
                        elapsed = 0.0

                elif state == "WIN":
                    if event.key == pygame.K_r:
                        load_level_idx(0)
                        particles.items.clear()
                        player.hp = player.max_hp
                        state = "PLAY"
                        paused = False
                        elapsed = 0.0

                elif state == "UPGRADE":
                    if event.key in (pygame.K_1, pygame.K_2, pygame.K_3):
                        idx = {pygame.K_1: 0, pygame.K_2: 1, pygame.K_3: 2}[event.key]
                        upg_id, _ = upgrade_choices[idx]
                        apply_upgrade(player, upg_id)

                        nxt = level_idx + 1
                        if nxt >= len(levels):
                            state = "WIN"
                        else:
                            load_level_idx(nxt)
                            state = "PLAY"
                            paused = False

        keys = pygame.key.get_pressed()

        # -------- Update --------
        if state == "PLAY" and not paused:
            level.update(dt)

            # player
            player.update(dt, level, keys, bullets, particles)

            # bullets
            for b in bullets:
                b.update(dt, level)
            bullets = [b for b in bullets if b.life > 0]

            for b in enemy_bullets:
                b.update(dt, level)
            enemy_bullets = [b for b in enemy_bullets if b.life > 0]

            # enemies
            for e in enemies:
                e.update(dt, level)

            # boss
            if boss is not None and boss.alive:
                boss.update(dt, level, enemy_bullets, player.rect)

            # player bullets hit enemies/boss
            for b in bullets:
                if b.owner != "player":
                    continue
                br = b.rect
                for e in enemies:
                    if e.hp > 0 and br.colliderect(e.rect):
                        e.damage(b.dmg)
                        b.life = 0
                        particles.burst(br.centerx, br.centery, n=12, color=PINK, speed=260, radius=3)
                        cam.shake = max(cam.shake, 5.0)
                        break
                if b.life <= 0:
                    continue
                if boss is not None and boss.alive and br.colliderect(boss.rect):
                    boss.damage(b.dmg)
                    b.life = 0
                    particles.burst(br.centerx, br.centery, n=16, color=(170, 90, 255), speed=300, radius=3.5)
                    cam.shake = max(cam.shake, 7.0)

            # cleanup dead enemies
            alive = []
            for e in enemies:
                if e.hp <= 0:
                    particles.burst(e.rect.centerx, e.rect.centery, n=18, color=RED, speed=280, radius=3)
                else:
                    alive.append(e)
            enemies = alive

            # boss defeated
            if boss is not None and (not boss.alive) and (not boss_defeated):
                boss_defeated = True
                particles.burst(boss.rect.centerx, boss.rect.centery, n=80, color=(200, 120, 255), speed=420, radius=4)
                cam.shake = max(cam.shake, 14.0)

            # enemy contact damage
            for e in enemies:
                if player.rect.colliderect(e.rect):
                    if player.hurt(1):
                        cam.shake = max(cam.shake, 9.0)
                        particles.burst(player.rect.centerx, player.rect.centery, n=16, color=RED, speed=260, radius=3)
                        player.vx += sign(player.rect.centerx - e.rect.centerx) * 280
                        player.vy = -280

            # boss bullets hit player
            for b in enemy_bullets:
                if b.rect.colliderect(player.rect):
                    b.life = 0
                    if player.hurt(1):
                        cam.shake = max(cam.shake, 10.0)
                        particles.burst(player.rect.centerx, player.rect.centery, n=18, color=(170, 90, 255), speed=300, radius=3)
                        player.vx += sign(player.rect.centerx - b.x) * 220
                        player.vy = -240

            # spikes damage
            for sr in level.spike_rects_near(player.rect):
                if player.rect.colliderect(sr):
                    if player.hurt(1):
                        cam.shake = max(cam.shake, 10.0)
                        particles.burst(player.rect.centerx, player.rect.bottom, n=18, color=RED, speed=300, radius=3)
                        player.vy = -460

            # coins collect
            for (pos, cr) in level.coin_rects_near(player.rect):
                if player.rect.colliderect(cr) and pos in level.coins:
                    level.coins.remove(pos)
                    coins_left -= 1
                    particles.burst(cr.centerx, cr.centery, n=14, color=YELLOW, speed=250, radius=3)
                    cam.shake = max(cam.shake, 3.0)

            # door check
            dr = level.door_rect()
            if dr and door_open() and player.rect.colliderect(dr):
                if level_idx == len(levels) - 1:
                    state = "WIN"
                else:
                    upgrade_choices = pick_3_upgrades(rng)
                    state = "UPGRADE"
                    paused = False

            # game over
            if player.hp <= 0:
                state = "GAMEOVER"

            particles.update(dt)
            cam.update(player.rect, level.world_w, level.world_h, dt)

        else:
            particles.update(dt * 0.7)

        # -------- Draw --------
        screen.fill(BG)

        if state == "TITLE":
            if random.random() < 0.06:
                particles.burst(random.randint(0, WIDTH), random.randint(0, HEIGHT), n=2, color=NEON, speed=40, radius=2)
            draw_text(screen, big, "NEON RUNNER — ADVANCED", WIDTH // 2, HEIGHT // 2 - 80, NEON, center=True)
            draw_text(screen, mid, "A/D move • W/SPACE jump • Shift dash • J shoot", WIDTH // 2, HEIGHT // 2 - 18, WHITE, center=True)
            draw_text(screen, font, "Wall-jump + Double-jump + Moving Platforms + Final Boss", WIDTH // 2, HEIGHT // 2 + 16, (180, 200, 255), center=True)
            draw_text(screen, font, "Press ENTER to start", WIDTH // 2, HEIGHT // 2 + 70, WHITE, center=True)
            particles.draw(screen, Camera())

        else:
            # world
            level.draw(screen, cam, door_open(), boss_alive=(boss is not None and boss.alive))
            for b in bullets:
                b.draw(screen, cam)
            for b in enemy_bullets:
                b.draw(screen, cam)
            for e in enemies:
                e.draw(screen, cam)
            if boss is not None:
                boss.draw(screen, cam)
            player.draw(screen, cam)
            particles.draw(screen, cam)

            if state == "PLAY":
                draw_hud(screen, font, player, level.name, coins_left, boss if boss_required else None, paused=paused)
                if paused:
                    overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
                    overlay.fill((0, 0, 0, 140))
                    screen.blit(overlay, (0, 0))
                    draw_text(screen, big, "PAUSED", WIDTH // 2, HEIGHT // 2 - 20, WHITE, center=True)
                    draw_text(screen, font, "Press ESC to resume", WIDTH // 2, HEIGHT // 2 + 30, WHITE, center=True)

            elif state == "UPGRADE":
                overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
                overlay.fill((0, 0, 0, 160))
                screen.blit(overlay, (0, 0))
                draw_text(screen, big, "CHOOSE AN UPGRADE", WIDTH // 2, 90, NEON, center=True)
                draw_text(screen, font, "Press 1 / 2 / 3", WIDTH // 2, 135, WHITE, center=True)

                for i, (uid, name) in enumerate(upgrade_choices):
                    cx = WIDTH // 2
                    cy = 220 + i * 90
                    card = pygame.Rect(0, 0, 760, 64)
                    card.center = (cx, cy)
                    pygame.draw.rect(screen, (20, 22, 34), card, border_radius=14)
                    pygame.draw.rect(screen, WHITE, card, 2, border_radius=14)
                    draw_text(screen, mid, f"{i+1}. {name}", card.centerx, card.centery - 2, YELLOW, center=True)

            elif state == "GAMEOVER":
                overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
                overlay.fill((0, 0, 0, 170))
                screen.blit(overlay, (0, 0))
                draw_text(screen, big, "GAME OVER", WIDTH // 2, HEIGHT // 2 - 40, RED, center=True)
                draw_text(screen, font, "Press R to restart", WIDTH // 2, HEIGHT // 2 + 20, WHITE, center=True)

            elif state == "WIN":
                overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
                overlay.fill((0, 0, 0, 170))
                screen.blit(overlay, (0, 0))
                draw_text(screen, big, "YOU WIN!", WIDTH // 2, HEIGHT // 2 - 70, NEON, center=True)
                draw_text(screen, mid, f"Total time: {elapsed:.1f}s", WIDTH // 2, HEIGHT // 2 - 14, WHITE, center=True)
                draw_text(screen, font, "Press R to play again", WIDTH // 2, HEIGHT // 2 + 40, WHITE, center=True)

        pygame.display.flip()

# --- Choose which levels to load ---
# JSON levels created earlier:
level_paths = [os.path.join("levels", f"level{i}.json") for i in (1,2,3)]

# Or load a TMX level (requires pytmx), e.g.:
# level_paths = ["levels/my_level1.tmx", "levels/my_level2.tmx"]

main(level_paths)
